In [ ]:
import numpy as np
import pandas as pd

# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt
from scipy import stats

# We will use the Seaborn library
import seaborn as sns
sns.set()

#precisao de casas decimais
pd.set_option("display.precision", 3)
#maximo de colunas
pd.set_option("display.max_columns", 200)
#maximo de linhas
pd.set_option("display.max_rows", 100)

In [ ]:
df = pd.read_csv('./incident_event_log.csv', low_memory=False)

In [ ]:
#ajustando notify
df['notify'] = df['notify'].map({'Do Not Notify': False, 'Send Email': True})
#ajustando datas
df['opened_at'] = pd.to_datetime(df['opened_at'])
df['sys_created_at'] = pd.to_datetime(df['sys_created_at'])
df['sys_updated_at'] = pd.to_datetime(df['sys_updated_at'])
df['resolved_at'] = pd.to_datetime(df['resolved_at'])
df['closed_at'] = pd.to_datetime(df['closed_at'])

In [ ]:
#diferença entre abertura e fechamento 
df['days_to_close'] = df['closed_at'] - df['opened_at']

In [ ]:
df.head(100)

In [ ]:
#tipos de abertura
df[df['sys_mod_count'] == 0].groupby(by='incident_state').size()

In [ ]:
#verificamos que possuem mais chamados fechados que chamados abertos

#len(pd.unique(df[df['sys_mod_count'] == 0]['number']))

df[df['sys_mod_count'] == 0].groupby(by='incident_state').size().sum() - len(df[df['incident_state'] == 'Closed'])

In [ ]:
#quantia de chamados com datas inválidas
len(df[df['days_to_close'] < pd.Timedelta(0,'m')])

In [ ]:
#quantia de chamados que iniciam com alterações
df[((df['sys_mod_count'] != 0) & (df['sys_created_at'] == df['sys_updated_at']))].groupby(by='number').size()

In [ ]:
#remover qualquer chamado com periodo para fechamento negativo
df = df[df['days_to_close'] > pd.Timedelta(0,'m')]

In [ ]:
#chamados fechados - mostrando que o ciclo de abertura e fechamento está incompleto
closed_df = df[df['incident_state'] == 'Closed']

In [ ]:
#descrição do período que levou para fechar os tickets.
closed_df['days_to_close'].describe()

In [ ]:
category_df = closed_df.groupby(by=['category', 'subcategory'])
category_df.size()

In [ ]:
#chamados mais recorrentes
category_df.size().sort_values(ascending=False).head(10)

In [ ]:
category_df['days_to_close'].describe()

In [ ]:
#selecionando colunas que se repetem nas linhas referentes ao mesmo incidente
common_columns = np.array(['number', 'caller_id', 'opened_by', 'opened_at', 'sys_created_by', 'sys_created_at', 'contact_type', 'notify', 'closed_code', 'resolved_at', 'closed_at'])
#agrupando em um novo df
grouped_df = df.groupby(by=list(common_columns))

In [ ]:
df.describe()

In [ ]:
df.describe(include=['object'])